# Ingesting Data

In python there are multiple ways to ingest data into a pandas dataframe.  This is because we often will work with data from various sources, even in a single project, and thus being able to easily ingest and format data is crucial to being productive.

We will be looking at 6 different ways to ingest data today:
- via csv
- via JSON
- via API
- via google sheets
- via SQL

## CSV ingestion

One of the most common data formats you'll deal with in the real world is csvs.  CSVs are an easy, text based representation of tabular data.  It can be easily saved (it's just a text file), passed around and while it's not the most efficient way to store data, it is less verbose than other formats like JSON. 

There are a few ways to read csv data into a pandas DataFrame.  The easiest way is to use `.read_csv`, e.g.:

In [ ]:
import pandas as pd

In [ ]:
market_caps = pd.read_csv('data/market_caps.csv')
market_caps.head()

We can also see that `.read_csv` will smartly try to format columns for us.  E.g. it can detect that MarketCap, Price and VolumeUSD are floats, and Rank is an integer

In [ ]:
market_caps.dtypes

However, we can add a bit more customization to this.  For example, let's say we wanted custom names and to use rank as an index.  We can do so directly from `.read_csv`:

In [ ]:
market_caps_custom = pd.read_csv(
    'data/market_caps.csv',
    header=None,
    skiprows=1,
    names=['rank', 'name', 'sym', 'market_size', 'price', 'volume_usd'],
    index_col=0
)
market_caps_custom.head()

## JSON ingestion

JSONs have similar properties to csvs, in that they are simple text files that can be passed around easily, however they can be a bit harder to work with for data analysis.  

While a JSON can represent a table, it can also represent any arbitrary bundle of data.  This means that we may need to parse the JSON first and extract out the relevant values before we can convert it into a pandas DataFrame.

Let's start with a simple example where the JSON is nicely formatted, and represents a table.  In this case, like with `.read_csv`, ad can simply call `.read_json`

In [ ]:
import json

In [ ]:
market_caps_json = pd.read_json('data/market_caps.json', orient='columns')

In [ ]:
market_caps_json

However, this won't work with a more complex json

In [ ]:
ts = pd.read_json('data/payload.json', orient='columns')
ts

This isn't what we're looking for.  What if we tried the other way:

In [ ]:
ts = pd.read_json('data/payload.json', orient='index')
ts

Unfortunately this doesn't help us either.  This is because in this case the JSON file is structured as a dictionary only, and not as a table.  Pandas will still try to read in the key/value pairs as a table and make things work, but in this case we won't be able to use it.

Instead, we will need to first get down to the key we care about, then pass that to pandas, i.e.:

In [ ]:
with open('data/payload.json', 'r') as file:
    json_data = json.load(file)

In [ ]:
(lambda x: x.assign(ts=pd.to_datetime(x.ts, unit='s')))(
    pd.DataFrame(
        json_data['result']['3600'],
        columns=['ts', 'open', 'high', 'low', 'close', 'volume', 'volumeUSD']
    )
)


## API Ingestion

In [ ]:
import requests
import pandas as pd

we've mostly been using API for data retrieval in this course.  The majority of the time you'll be using the request package.  Here is what we have been using in class so far, but lets dig into what this means a little more

In [ ]:
res = requests.get(
    f'https://api.cryptowat.ch/markets/coinbase-pro/btcusd/ohlc',
    params={
        'periods': '3600',
        'after': str(int(pd.Timestamp('2021-12-01').timestamp()))
    }
)

most APIs that we will hit will be REST APIs (http based APIs). For these there are two types of calls that you can do, either `get` requests or `post` requests.  The main difference is that `get` requests will take in parameters directly in the URL, while `post` parameters will allow you to pass in data (usually called a payload) into request.

For many apis you will also need to pass in an `Authorization` header if it needs authentication.

Once we have called the API with our request, we will get a response.  Firstly, we can check if the response is successful by looking at the status code.  Usually REST APIs will denote success with a status code of `200`, and a status code of `400`, `500` or most other codes will mean failure

In [ ]:
res.status_code

We can see above that the status code is successful, so now we can take a look at the content of our response

In [ ]:
res.text

as you can see, all responses that is passed back from an API call can be read as text.  However in our case, we know ahead of time that the response body is a json, and so we can use `.json` method in the response object to convert the text into a python dict, which would be much easier for us to use

In [ ]:
res.json()

finally, we can use the same technique as the JSON section and pull the relevant information we need from the JSON and load it into a DataFrame

In [ ]:
(lambda x: x.assign(ts=pd.to_datetime(x.ts, unit='s')))(
    pd.DataFrame(
        res.json()['result']['3600'],
        columns=['ts', 'open', 'high', 'low', 'close', 'volume', 'volumeUSD']
    )
)


## Google sheets ingestion

In [ ]:
import os
import pandas as pd
import gspread

another useful source for ingestion is google sheets.  While you an ingest larger data sets, where google sheets really shines is for getting tabular information for summary data, or especially configs.

A good example of this is running a model where certain parameters (e.g. priors for coefficients) are set by domain experts, while the model is developed by data scientists and will take these priors as an input to generate the final result.  In that situation the domain experts probably work much better in a spreadsheet setting, while the data scientists will prefer code.  We can leverage google sheets to allow the domain experts to enter and update the coefficients, then we can programatically read the values and join against the rest of our data.

Example usage for google sheets using the library gspread is below, and you can find the setup information here: https://github.com/burnash/gspread

In [ ]:
gc = gspread.service_account()

In [ ]:
sheet = gc.open('market_caps').worksheet('Sheet1')

In [ ]:
data = sheet.get_all_records()

In [ ]:
pd.DataFrame(data)

## SQL Ingestion

one of the most common ways we will be accessing data is via SQL.  

For our examples, we will look at how to access a sql database, read, write and join it using sqlite

## SQLite

SQLite is a powerful, file-based sql database.  It's build as a local database for single reader/writer functionality, and has most of the common SQL functionality out of the box.  We can learn about most python<>sql interactions with SQLite as the interaction patterns carries over to all flavors of sql.

The two key objects for interacting with any SQL database are the `connection` and the `cursor`.  The connection represents a connection to a SQL database, while the cursor represents an execution object for that connection.  For example:

In [ ]:
import sqlite3
import pandas as pd

%load_ext sql

In [ ]:
conn = sqlite3.connect('data/data.db')

in the above, we create a connection to the SQL database at `data/data.db`.  The connection object can save changes with `.commit` or close the connection with `.close`.  We can also create a cursor object from the connection to use for querying:

In [ ]:
cur = conn.cursor()

Now that we have the cursor object, we can start executing SQL queries.  For example, let's take a look at what tables we have in this database:

In [ ]:
for row in cur.execute('SELECT name FROM sqlite_master'):
    print(row)

or even the schema itself:

In [ ]:
for row in cur.execute('SELECT name, sql FROM sqlite_master'):
    print(row)

Most of the time though, we want to pull information from the database.  For example, let's pull some data from the `ohlc` table:

In [ ]:
for row in cur.execute("SELECT * FROM ohlc WHERE token = 'BTC' LIMIT 5"):
    print(row)

we can also also get all the data at once instead of getting data row by row:

In [ ]:
cur.execute("SELECT * FROM ohlc WHERE token = 'BTC' LIMIT 5")
data = cur.fetchall()
data

And because it's a list of tuples, we can simply create a pandas DataFrame and add in columns to get our data

In [ ]:
pd.DataFrame(
    data,
    columns=['ts', 'open', 'high', 'low', 'close', 'volume', 'volumeUSD', 'token', 'chain']
)

finally, we can close the connection

In [ ]:
conn.close()

after closing the connection, as expected the cursor will stop working, and no queries can be executed

In [ ]:
cur.execute("SELECT * FROM ohlc WHERE token = 'BTC' LIMIT 5")
cur.fetchall()

## SQL Basics

SQL is a very flexible language, and most data analysis tools (including pandas) borrows a lot of core concepts that SQL has when working with data.  In this section we'll look at a few basic SQL functions that will help us create the datasets that we need before we pull them to python side to work on them.

Firstly, let's look at basic querying.  We can query for all columns using `*`, or select the specific columns.  The general syntax is `SELECT [columns | *] FROM [table_name] WHERE [conditions...]`

In [ ]:
%%sql sqlite:///data/data.db

SELECT * FROM ohlc WHERE token = 'BTC' LIMIT 10

In [ ]:
%%sql

SELECT ts, token, close FROM ohlc WHERE token = 'BTC' LIMIT 10

In addition, we can also aggregate very easily using the `GROUP BY` clause

In [ ]:
%%sql

SELECT 
    token, 
    SUM(volumeUSD) / 1e9 AS volumeUSD, 
    SUM(volume) AS volume, 
    AVG(close) AS avg_close 
FROM ohlc 
GROUP BY 1 
ORDER BY 2 DESC

In addition, like with pandas we can do joins.  The notation is shown below:

In [ ]:
%%sql 

SELECT
    ts,
    token,
    close,
    MarketCap
FROM
    ohlc
    JOIN market_caps ON ohlc.token = market_caps.Symbol
LIMIT 20

we can also get much fancier with our querying, using CTEs (common table expressions) and joins

In [ ]:
%%sql

WITH

volume_summaries AS (
SELECT 
    token, 
    SUM(volumeUSD) / 1e9 AS volumeUSD
FROM ohlc 
GROUP BY 1 
ORDER BY 2 DESC
),

token_caps AS (
SELECT
    Symbol AS token,
    MarketCap / 1e9 AS market_cap
FROM
    market_caps
)

SELECT
    *,
    ROUND(100 * volumeUSD / market_cap) AS float_traded_total
FROM
    token_caps
    JOIN volume_summaries USING(token)

**note** for the magic function that we used `%%sql`, we can actually convert the data to a DataFrame instead of just displaying it on output

In [ ]:
%%sql result <<

SELECT
    ts,
    token,
    close,
    MarketCap
FROM
    ohlc
    JOIN market_caps ON ohlc.token = market_caps.Symbol
LIMIT 20

In [ ]:
result.DataFrame().head()

## SQL and Pandas

We saw above how to use cursors to access the database, fetch data with SQL queries and convert the result to a dataframe.  However, we can do this just as easily with pandas directly using the `.read_sql` method.

In [ ]:
pd.read_sql("SELECT * FROM ohlc WHERE token = 'BTC' LIMIT 10", 'sqlite:///data/data.db')

or if we don't want ot open/close the connection everytime, we can also create a connection first, then pass in the connection

In [ ]:
conn = sqlite3.connect('data/data.db')

In [ ]:
pd.read_sql("SELECT * FROM ohlc WHERE token = 'BTC' LIMIT 10", conn)

Since the data coming back from sql will always be in a tabular form and will have type information for each column, this means that the data set will (almost) always convert nicely into a pandas dataframe.  In fact, we can also turn the schema lookup into a table:

In [ ]:
schema = pd.read_sql('SELECT name, sql FROM sqlite_master', conn)
schema

In [ ]:
schema.sql[0]

We can actually use the queries above as is, and load it directly into a pandas DataFrame:

In [ ]:
summaries = pd.read_sql('''
WITH

volume_summaries AS (
SELECT 
    token, 
    SUM(volumeUSD) / 1e9 AS volumeUSD
FROM ohlc 
GROUP BY 1 
ORDER BY 2 DESC
),

token_caps AS (
SELECT
    Symbol AS token,
    MarketCap / 1e9 AS market_cap
FROM
    market_caps
)

SELECT
    *,
    ROUND(100 * volumeUSD / market_cap) AS float_traded_total
FROM
    token_caps
    JOIN volume_summaries USING(token)
''', conn)
summaries

We can also write back to our SQL database very easily, using `.to_sql`.  This allows us to process data in python, and once we're ready we can now save the results back down into a SQL database (which can be used for other people for example, or used by us later if we want to join against other datasets)

In [ ]:
summaries.to_sql('summaries', conn, if_exists='fail')

notice that we set `if_exists` equal to `fail` (this is also the default option).  This prevents accidental overwrites, as if we try to run the command again it will detect the table already existing and fail.  If this is not the desired behavior we can also use `replace` or `append` options, which will either rewrite the table everytime, or append the data to the bottom.

In [ ]:
summaries.to_sql('summaries', conn, if_exists='replace')

lastly, if we want to drop any tables, we can just use a cursor and issue a drop command

In [ ]:
conn.cursor().execute('DROP TABLE summaries')
conn.commit()